In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import const
from romar import backend as bkd
from romar.systems import BoxIso

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database"

In [5]:
model = BoxIso(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  rad_dtb=path_to_dtb + "/rates/rad_fit.p",
  use_rad=True,
  use_coll_int_fit=True
)

In [6]:
sol_ref = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/argon/testcase/test_iso/output_box"

In [ ]:
columns = "t X_em X_Ar X_Arp Th Te rho p n E".split()
box = np.loadtxt(sol_ref + "/box.dat")
box = pd.DataFrame(box, columns=columns)
box

In [8]:
t = box["t"].values
rho, Th0, Te0 = [box[k][0] for k in ("rho", "Th", "Te")]

In [9]:
def read_pop(name):
  ni_gi  = np.loadtxt(sol_ref + f"/pop_{name}.dat", comments="&", skiprows=2)[:,1]
  levels = model.mix.species[name].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [10]:
n = {k: read_pop(k) for k in ("Ar", "Arp")}
n["em"] = (box["X_em"] * box["n"]).values.reshape(-1,1)
n_true = np.hstack([n[k] for k in model.species_order]).T
rho_true = model.mix.get_rho(bkd.to_torch(n_true)).numpy()
T_true = box[["Th", "Te"]].values.T
rho = np.sum(rho_true[:,0])

In [ ]:
y0 = np.zeros(model.nb_eqs+2)
y0[:model.mix.nb_comp] = rho_true[:,0] / rho
y0[-2] = Th0
y0[-1] = model.mix.get_pe(Te=Te0, ne=n_true[-1,0])
y0

In [ ]:
y, runtime = model.solve_fom(t, y0, rho)
runtime

In [13]:
w_pred = copy.deepcopy(y[:model.mix.nb_comp])
n_pred = model.mix.get_n(bkd.to_torch(w_pred)).numpy()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  for i in s.indices:
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  plt.show()
  plt.close()

Linear model

In [15]:
def solve_lin(t, y0):
  A = model.jac(0.0, y0)
  b = model.fun(0.0, y0)
  y = sp.integrate.solve_ivp(
    fun=lambda t, y: A @ y + b,
    t_span=[0.0,t[-1]],
    y0=np.zeros_like(y0),
    method="LSODA",
    t_eval=t,
    first_step=1e-14,
    rtol=1e-6,
    atol=1e-20,
    jac=lambda t, y: A
  ).y
  y += y0.reshape(-1,1)
  return y

In [16]:
ylin = solve_lin(t, y0[:-2])

In [ ]:
for i, yi in enumerate(y.T):
  A = model.jac(0.0, yi)
  l, v = sp.linalg.eig(A.T)
  stable = (l.real < 0.0).all()
  if not stable:
    print(np.where(l.real >= 0.0))
    print(t[i], l.real.max())

In [18]:
w_lin = copy.deepcopy(ylin[:model.mix.nb_comp])
n_lin = model.mix.get_n(bkd.to_torch(w_lin)).numpy()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_pred[i] - n_lin[i])/n_pred[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_pred[i][1:])
    plt.loglog(t[1:], n_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()